In [1]:
# load in data from HPI
from my.location.via_ip import locations
locs = sorted(locations(), key=lambda l: l.dt)


/home/sean/Repos/HPI-karlicoss/my/location/via_ip.py:7: UserWarning: my.location.via_ip is deprecated, use my.location.fallback.via_ip instead
  high("my.location.via_ip is deprecated, use my.location.fallback.via_ip instead")
/home/sean/Repos/HPI-karlicoss/my/location/fallback/via_ip.py:55: UserWarning: locations is deprecated, should use fallback_locations or estimate_location
  medium("locations is deprecated, should use fallback_locations or estimate_location")


In [2]:
# per day info
import pandas as pd
from itertools import groupby
locs_sample = []
for d, ll in groupby(locs, key=lambda l: l.dt.date().day):
    res, count = Counter(list(ll)).most_common(1)[0]
    locs_sample.append(res)
locs_sample = pd.DataFrame(locs_sample)
# print(len(locs_sample)) # (number of days data exists for)

In [4]:
# https://wiki.openstreetmap.org/wiki/Mercator#Python_implementation
import math

def merc_x(lon):
  r_major=6378137.000
  return r_major*math.radians(lon)

def merc_y(lat):
  if lat>89.5:lat=89.5
  if lat<-89.5:lat=-89.5
  r_major=6378137.000
  r_minor=6356752.3142
  temp=r_minor/r_major
  eccent=math.sqrt(1-temp**2)
  phi=math.radians(lat)
  sinphi=math.sin(phi)
  con=eccent*sinphi
  com=eccent/2
  con=((1.0-con)/(1.0+con))**com
  ts=math.tan((math.pi/2-phi)/2)/con
  y=0-r_major*math.log(ts)
  return y

import math
# ugh https://gis.stackexchange.com/a/156046 this one works fine??
def merc(lat, lon):
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

# meh
mx = []
my = []
for lat, lon in zip(locs_sample['lat'], locs_sample['lon']):
    (mmx, mmy) = merc(lat, lon)
    mx.append(mmx)
    my.append(mmy)
locs_sample['merc_lat'] = mx
locs_sample['merc_lon'] = my

In [ ]:
# https://github.com/karlicoss/dashboard/blob/d750b9322574d136055acc8e26065580856b1fab/notebooks/bokeh_test.ipynb

from bokeh.plotting import figure, show
from bokeh.tile_providers import OSM, get_provider
from bokeh.models import ColumnDataSource as CDS

tile_provider = get_provider(OSM)

x_range = min(int(locs_sample['merc_lat'])), int(max(locs_sample['merc_lat']))
y_range = min(int(locs_sample['merc_lon'])), int(max(locs_sample['merc_lon']))

#print(x_range, y_range)

p = figure(title='map', x_axis_type="mercator", y_axis_type="mercator", plot_width=2000, x_range=x_range, y_range=y_range)
p.add_tile(tile_provider)

source = CDS(data=locs_sample)

p.circle(x="merc_lat", y="merc_lon", size=15, fill_color="blue", fill_alpha=0.8, source=source)

#save to html file
#output_file("file.html")
#save(p)
show(p)
#print(locs_df)